In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
import numpy as np
import requests
from config import rapid_api_key
from config import rapidapi_host
from pprint import pprint
import datetime

In [ ]:
# os.getcwd()
# os.chdir('C:\\gitlab\\covid19')

In [ ]:
county_path = 'Resources/us_counties_states.csv'
date_path = 'Resources/dates.csv'

In [ ]:
us_counties_df = pd.read_csv(county_path)
mask1 = us_counties_df['State'] == 'California'
mask2 = us_counties_df['State'] == 'Indiana'

us_counties_df = us_counties_df[mask1 | mask2]
us_counties_df.head()

In [ ]:
date_df = pd.read_csv(date_path)
date_df.head()

In [ ]:
county_list = []
state_list = []
county_list = us_counties_df['County, State'].tolist()
date_list = date_df['Date'].tolist()

In [ ]:
date = "2020-10-10"
url = "https://covid-19-statistics.p.rapidapi.com/reports"
headers = {'x-rapidapi-host': rapidapi_host,'x-rapidapi-key': rapid_api_key}

# State lists
new_state = []
total_cases_state = []
new_cases_state = []
tot_deaths_state = []
new_deaths_state = []
fatality_rate_state = []
date_state = []

# County Lists
county_state_list = []
new_county_list = []
tot_county_cases = []
new_county_cases = []
tot_county_deaths = []
new_county_deaths = []
county_date = []
county_lat = []
county_lng = []


for date_val in date_list:
    try:
        record_counter = 0
        set_counter = 1

        for name in county_list:
            value = name.split(',')
            county = value[0]
            state = value[1]
            country_state = "US " + state
            record_counter +=1

            querystring = {"region_province":state,"iso":"USA","region_name":"US","city_name":county, \
                           "date":date,"q":country_state}   

            response = requests.request("GET", url, headers=headers, params=querystring).json()    

            try:
                # Add State Information
                new_state.append(response['data'][0]['region']['province'])
                total_cases_state.append(response['data'][0]['confirmed'])
                new_cases_state.append(response['data'][0]['confirmed_diff'])
                tot_deaths_state.append(response['data'][0]['deaths'])
                new_deaths_state.append(response['data'][0]['deaths_diff'])
                fatality_rate_state.append(response['data'][0]['fatality_rate'])
                date_state.append(response['data'][0]['date_val'])         

                # Add County information
                county_state_list.append(response['data'][0]['region']['province'])
                new_county_list.append(response['data'][0]['region']['cities'][0]['name'])
                tot_county_cases.append(response['data'][0]['region']['cities'][0]['confirmed'])
                new_county_cases.append(response['data'][0]['region']['cities'][0]['confirmed_diff'])
                tot_county_deaths.append(response['data'][0]['region']['cities'][0]['deaths'])
                new_county_deaths.append(response['data'][0]['region']['cities'][0]['deaths_diff'])
                county_date.append(response['data'][0]['region']['cities'][0]['date_val'])
                county_lat.append(response['data'][0]['region']['cities'][0]['lat'])
                county_lng.append(response['data'][0]['region']['cities'][0]['long'])

                if record_counter > 150:
                    record_counter = 1
                    set_counter += 1

                print(f'Processing record {record_counter} of set {set_counter} | {county}, {state}, {date_val}.')

            except:
                record_counter -= 1
                print(f'Error: County: {county} and State:{state} not found')
                
        set_counter += 1
        
    except:
        print(f'Date Error: {date_val} was not processed')
            



In [ ]:
print(len(new_state))
print(len(county_state_list))
print(len(county_state_list))
print(len(new_county_list))
print(len(tot_county_cases))
print(len(new_county_cases))
print(len(tot_county_deaths))
print(len(new_county_deaths))
print(len(county_date))
print(len(county_lat))
print(len(county_lng))

In [ ]:
covid_state_df = pd.DataFrame({
    "State" : new_state,
    "Total Cases-State" : total_cases_state,
    "New Cases-State" : new_cases_state,
    "Total Deaths-State" : tot_deaths_state,
    "New Deaths-State" : new_deaths_state,
    "Fatality Rate-State" : fatality_rate_state,
    "Date-State" : date_state    
})    

# covid_state_df.drop_duplicates(keep='first', inplace=True)
covid_state_df=covid_state_df.sort_values(by="Date-State", ascending=False)
# covid_state_df.to_csv('Resources/state_df.csv')


In [ ]:
covid_county_df = pd.DataFrame({
    "State-County" : county_state_list,
    "County" : new_county_list,
    "Total Cases-County" : tot_county_cases,
    "New Cases-County" : new_county_cases,
    "Total Deaths-County" : tot_county_deaths,
    "New Deaths-County" : new_county_deaths,
    "Date-County" : county_date,
    "Latitude-County" : county_lat,
    "Longitude-County" : county_lng   
})


covid_county_df

In [ ]:
day_change = datetime.timedelta(days=1)
end_date = datetime.date.today()
start_date = datetime.datetime.today() - datetime.timedelta(days=224)
nextday_date = datetime.datetime.today() + datetime.timedelta(days=1)